In [2]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
# One-hot encoding categorical columns
dataset = pd.get_dummies(dataset, drop_first=True)


In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [7]:
dataset=dataset.drop("User ID",axis=1)

In [8]:
dataset["Purchased"].value_counts() # Here we have to count the o's and 1's 
#Here we can check this BALANCED/IMBALANCED?
#If Balanced Dataset = It'll be in Equal numbers for both 0 & 1
# If Imbalanced = Deviations will be there.
# Unique Entries using .value_counts() method

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
indep=dataset[['Age','EstimatedSalary','Purchased','Gender_Male']]
dep=dataset["Purchased"] # Here as usual we are not splitting the training set and test for Regression - GRID purpose.
#But here in the classification Grid we should spilt the training set and test set.
#If we not split the same Report will generate.  

In [10]:
indep.shape

(400, 4)

In [11]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [12]:
# Split into training set and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [13]:
#Here We should add the Standard Scaler option as well

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() #if our predcitions will be under -2 to +2, Then we can keep it in the Standard Scaler
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
# htpps://scikit-learn.org/stable/modules/model_evaluation.html#storing-parameter

param_grid = {
    'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
    'penalty':['l2']   # 👈 typo here
}

grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3, n_jobs = -1, scoring = 'f1_weighted') #It will run automatically and take the best model from the grid list
# We can do with RECALL & PRecision as well for the prediction(Which model is giving better performance)
# fitting the model for grid search
grid.fit(X_train, y_train)


Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [20]:
'''
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {
    "C": [0.1, 1, 10],                 # Regularization strength
    "solver": ["liblinear", "lbfgs"],  # Compatible solvers
    "penalty": ["l2"],                 # l1 works only with liblinear or saga
    "max_iter": [500]                  # Increase iterations
}

grid = GridSearchCV(
    LogisticRegression(),
    param_grid,
    refit=True,
    verbose=3,
    n_jobs=-1,
    scoring="f1_weighted"
)

grid.fit(X_train, y_train)
'''

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'max_iter': [500],
                         'penalty': ['l2'], 'solver': ['liblinear', 'lbfgs']},
             scoring='f1_weighted', verbose=3)

In [23]:
# Print the best parameter after tuning
# print (grid.best_params_)
re=grid.cv_results_
# print(re)
grid_predictions = grid.predict(X_test) # we can store the X_test in to gird.prediction

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

# print classification report
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [24]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test, grid_predictions, average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_), f1_macro)

The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'newton-cg'}: 1.0


In [25]:
print("The Confusion Matrix:\n", cm)

The Confusion Matrix:
 [[85  0]
 [ 0 49]]


In [26]:
print("The Report:\n", clf_report)

The Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        85
           1       1.00      1.00      1.00        49

    accuracy                           1.00       134
   macro avg       1.00      1.00      1.00       134
weighted avg       1.00      1.00      1.00       134



In [29]:
from sklearn.metrics import roc_auc_score # [ROC = Receiver Operating Charactersitic Curve], [AUC = Area Under Curve]
roc_auc_score(y_test, grid.predict_proba(X_test)[:,1]) # proba => Parabola.
# where [:,1] => All rows in the first column
# : => represents the rows [We can take entire rows]
# And the 1 => Represents Column [first columns] 

np.float64(1.0)

In [30]:
table = pd.DataFrame.from_dict(re)

In [31]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007600,0.001577,0.005949,0.002283,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.007481,0.002675,0.011367,0.005895,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
2,0.004760,0.001370,0.004958,0.001103,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.004700,0.003367,0.003368,0.000746,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1


In [32]:
UserID_input=int(input("UserID:"))
Age_input=float(input("age:"))
EstimatedSalary_input=float(input("Estimated Salary:"))
Gender_Male_input=int(input("Gender Male 0 or 1:"))

UserID: 15668575
age: 26
BMI: 43000
Gender Male 0 or 1: 0


In [33]:
Future_Prediction=grid.predict([[UserID_input, Age_input, EstimatedSalary_input, Gender_Male_input]]) # Change the parameter according to your dataset
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


In [ ]:
#Save our model 
import pickle
filename="finalized_model_knn_classification.sav" # file name has been created.

In [ ]:
# Here we have to insert the already created model.
pickle.dump(classifier,open(filename,'wb')) # Here the 'wb' is "Write Binary".

# With this we have saved the model.

In [ ]:
# Next step is to load the saved model.
loaded_model=pickle.load(open("finalized_model_knn_classification.sav",'rb'))
#Here we have to check with the user input in real time.
result=loaded_model.predict([[15624510, 1, 19, 19000]])

In [ ]:
result